# Compute on COMEXT

## Import

In [1]:
import os, sys

colab = False
# #### COLAB ONLY CELL
if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    # drive.mount('/content/gdrive',force_remount=True)
    os.chdir("/content/gdrive/MyDrive/COMEXT")
    !pip install -q -r world-trade-thesis/requirements.txt
    !pwd
else:
    os.chdir("H:/My Drive/COMEXT")
    !pwd
import pandas as pd
from tqdm.notebook import tqdm

if colab:
    sys.path.append('/content/gdrive/MyDrive/COMEXT/world-trade-thesis/src')
else:
    sys.path.append('H:/My Drive/COMEXT/world-trade-thesis/src')
import graph_analysis as g

/h/My Drive/COMEXT
Functions loaded!


In [2]:
# import hashlib, random, math, os, sys, pickle, time, zipfile, io, json#, powerlaw #,eurostat
import pandas as pd
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm, trange
# from networkx.classes.function import degree
# import matplotlib.pyplot as plt
# from matplotlib import cm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Compute on whole dataset

In [3]:
# print("Running on whole time and merging to big df...")
# metrics_full = {}
# df_pop, eu_iso = g.load_population_df()
# big_df_col = ["year","month","prod","country"] +  ['edges','nodes','size','degree',
#  'degree_perc',
#  'out_degree',
#  'out_degree_perc',
#  'in_degree',
#  'in_degree_perc',
#  'density',
#  'density_adj',
#  'clustering',
#  'page_rank',
#  'vulnerability',
#  'in_weight_degree',
#  'out_weight_degree',
#  'degree_gamma',
#  'degree_xmin',
#  'in_degree_gamma',
#  'in_degree_xmin',
#  'out_degree_gamma',
#  'out_degree_xmin',
#  'in_weight_degree_gamma',
#  'in_weight_degree_xmin',
#  'out_weight_degree_gamma',
#  'out_weight_degree_xmin',
#  'out_weight_abs',
#  'in_weight_abs']

# big_df = pd.DataFrame(columns=big_df_col)
# issues = []
# for y,m in tqdm([(y,m) for y in range(2001,2022) for m in range(1,13)]):
#     # print(y,m)
#     df, _ = g.load_filtered_data("full", save=True, force_reload=False, verbose=False, sorted=False, columns=g.full_columns, types=g.full_types, group_by_prod_code=True, n_digits=2,
#                                 years=[y], months=[m], product="all", flow="all", declarant_iso="all", partner_iso="all", trade_type="all")
#     n4g = g.get_world_countries(df_pop, eu_iso, year=str(y))
#     for prod in df.PRODUCT_CPA2_1.unique():
#         df4g = df[df.PRODUCT_CPA2_1 == prod]
#         t4g = g.extract_table_for_graph(df4g, y=str(y), flow="all", scale_by='population', pop_df=df_pop)
#         if t4g.shape[0] < 10:
#             issues.append((y,m,prod))
#             # print(y,m,prod,t4g.shape[0])
#         coord, metrics, G = g.makeGraph(t4g, tab_nodes=n4g, weight_flag=True, criterio="QUANTITY_IN_KG")
#         x = metrics.reset_index().to_dict()
#         x["year"] = str(y)
#         x["month"] = str(m)
#         x["prod"] = prod
#         big_df = pd.concat([big_df,pd.DataFrame(x,columns=big_df_col)],axis=0)
#         metrics_full[f"{y}{m:02d}_cpa_{prod}"] = metrics #.sort_values("degree",ascending=False)
#     # break

# pickle.dump(metrics_full,open("./data-samples/manual/metrics/metrics_full_Q_v9.pickle","wb"))
# big_df.to_parquet("./data-samples/manual/metrics/metrics_full_Q_v9.parquet")


In [4]:
# print("Merging to big df...")
# big_df_col = ["year","month","prod","country"] + metrics_full["200101_cpa_TO"].columns.to_list()
# tot = 0
# big_df = pd.DataFrame(columns=big_df_col)
# for k in tqdm(metrics_full):
#     # tot += metrics_full[k].shape[0]
#     k_l = k.split("_")
#     x = metrics_full[k].rename({"index":"country"},axis=1).to_dict()
#     x["year"] = k_l[0][:4]
#     x["month"] = k_l[0][4:]
#     x["prod"] = k_l[2]
#     big_df = pd.concat([big_df,pd.DataFrame(x,columns=big_df_col)],axis=0)

In [5]:
# # metrics_full.keys()
# # metrics_full["200105_cpa_20"]
# df_to_combine = df[df.PRODUCT_CPA2_1 == "20"]
# df4g = df_to_combine[df_to_combine.PRODUCT_CPA2_1 == "TO"]
# df_eu = df4g[df4g.PARTNER_ISO.isin(df4g.DECLARANT_ISO.unique())]
# df_ex = df4g[~df4g.PARTNER_ISO.isin(df4g.DECLARANT_ISO.unique())]
# # df_eu
# df_to_avg = df_eu[df_eu.FLOW == 1].merge(df_eu[df_eu.FLOW ==2][["DECLARANT_ISO","PARTNER_ISO","VALUE_IN_EUROS","QUANTITY_IN_KG"]]
#                                         ,left_on=["DECLARANT_ISO","PARTNER_ISO"]
#                                         ,right_on=["PARTNER_ISO","DECLARANT_ISO"]
#                                         ,how="inner",suffixes=("","_y"))
# df_to_avg["VALUE_IN_EUROS"] = (df_to_avg["VALUE_IN_EUROS"] + df_to_avg["VALUE_IN_EUROS_y"])/2
# df_to_avg["QUANTITY_IN_KG"] = (df_to_avg["QUANTITY_IN_KG"] + df_to_avg["QUANTITY_IN_KG_y"])/2
# df_eu_l = [df_eu[df_eu.FLOW == 1].merge(df_eu[df_eu.FLOW ==2][["DECLARANT_ISO","PARTNER_ISO","VALUE_IN_EUROS","QUANTITY_IN_KG"]]
#                                         ,left_on=["DECLARANT_ISO","PARTNER_ISO"]
#                                         ,right_on=["PARTNER_ISO","DECLARANT_ISO"]
#                                         ,how="outer",suffixes=("","_y")
#                                         ,indicator=True).query('_merge=="left_only"')[df.columns]
#         ,df_to_avg[df.columns]
#         ,df_ex[df_ex.FLOW == 1]
#         ,df_ex[df_ex.FLOW == 2].rename({"DECLARANT_ISO":"PARTNER_ISO","PARTNER_ISO":"DECLARANT_ISO"},axis=1)]
# df_to_scale = pd.concat(df_eu_l)
# df_to_scale["country_from"] = df_to_scale.DECLARANT_ISO
# df_to_scale["country_to"] = df_to_scale.PARTNER_ISO
# df_to_scale

In [6]:
# TO ADJUST
# metrics_tr = {}
# for flow in [1,2]:
#     for y,m in tqdm([(y,m) for y in range(2010,2022) for m in range(1,13)]):
#         df, _ = load_filtered_data("tr", save=True, verbose=False, columns=tr_columns, types=tr_types, group_by_prod_code=True, n_digits=2,
#                                 years=[y], months=[m], product="all", flow="all", declarant_iso="all", partner_iso="all")
        
#         for prod in df.PRODUCT_NSTR.unique():
#             df4g = df[df.PRODUCT_NSTR == prod][tr_columns[1:]]
#             t4g = extract_table_for_graph(df4g, flow=flow, scale_by='population', pop_df=df_pop, threshold_cs=0.999)
#             coord, metrics, G = makeGraph(t4g, weight_flag=True, flow=[flow])
#             try:
#                 metrics_tr[f"{y}{m:02d}_nstr_{prod}_f{flow}"] = metrics.sort_values("degree",ascending=False)
#             except:
#                 pass

# pickle.dump(metrics_tr,open("./data-samples/manual/metrics/metrics_tr_10_21_v3.pickle","wb"))

In [7]:
# TO ADJUST
# metrics_tr_intra = {}
# for flow in [1,2]:
#     for y,m in tqdm([(y,m) for y in range(2010,2022) for m in range(1,13)]):
#         df, _ = load_filtered_data("tr_intra_", save=True, verbose=False, columns=tr_intra_columns, types=tr_types, group_by_prod_code=True, n_digits=2,
#                                 years=[y], months=[m], product="all", flow="all", declarant_iso="all", partner_iso="all")
        
#         for prod in df.PRODUCT_NSTR.unique():
#             df4g = df[df.PRODUCT_NSTR == prod][tr_intra_columns[1:]]
#             t4g = extract_table_for_graph(df4g, flow=flow, scale_by='population', pop_df=df_pop, threshold_cs=0.999)
#             coord, metrics, G = makeGraph(t4g, weight_flag=True, flow=[flow])
#             try:
#                 metrics_tr_intra[f"{y}{m:02d}_nstr_{prod}_f{flow}"] = metrics.sort_values("degree",ascending=False)
#             except:
#                 pass
# pickle.dump(metrics_tr_intra,open("./data-samples/manual/metrics_tr_intra_10_21_v3.pickle","wb"))

## WTO

In [8]:
# y = 2001
# df, _ = g.load_filtered_data("complete", save=True, force_reload=False, verbose=False, sorted=False, columns=g.full_columns, types=g.full_types, group_by_prod_code=True, n_digits=2,
#                                 years=[y], months=None, product="all", flow="all", declarant_iso="all", partner_iso="all", trade_type="all")

In [9]:
# big_df_col = ["year","month","prod","country"] +  ['edges','nodes','size','degree',
#     'degree_perc',
#     'out_degree',
#     'out_degree_perc',
#     'in_degree',
#     'in_degree_perc',
#     'density',
#     'density_adj',
#     'clustering',
#     'page_rank',
#     'vulnerability',
#     'in_weight_degree',
#     'out_weight_degree',
#     'degree_gamma',
#     'degree_xmin',
#     'in_degree_gamma',
#     'in_degree_xmin',
#     'out_degree_gamma',
#     'out_degree_xmin',
#     'in_weight_degree_gamma',
#     'in_weight_degree_xmin',
#     'out_weight_degree_gamma',
#     'out_weight_degree_xmin',
#     'out_weight_abs',
#     'in_weight_abs']

# big_df = pd.DataFrame(columns=big_df_col)
# metrics_full = {}
# issues = []
# df_pop, eu_iso = g.load_population_df()
# n4g = g.get_world_countries(df_pop, eu_iso, year=str(y))
# for prod in df.PRODUCT_CPA2_1.unique():
#     df4g = df[df.PRODUCT_CPA2_1 == prod]
#     t4g = g.extract_table_for_graph(df4g, y=str(y), flow="all", scale_by='population', pop_df=df_pop)
#     if t4g.shape[0] < 10:
#         issues.append((y,m,prod,t4g.shape[0]))
#         # print(y,m,prod,t4g.shape[0])
#     coord, metrics, G = g.makeGraph(t4g, tab_nodes=None, weight_flag=True, criterio="VALUE_IN_EUROS",compute_layout=True,lay_dist=20)
#     x = metrics.reset_index().to_dict()
#     x["year"] = str(y)
#     x["month"] = "-"
#     x["prod"] = prod
#     big_df = pd.concat([big_df,pd.DataFrame(x,columns=big_df_col)],axis=0)
#     metrics_full[f"{y}_cpa_{prod}"] = metrics #.sort_values("degree",ascending=False)
#     break

In [10]:
# plt.figure(figsize=(20,15),dpi=100)
# edg_color = [G.get_edge_data(u,v)["weight"] for u,v in G.edges()]
# m = max(edg_color)
# edg_color = [(i/m)*256 + 30 for i in edg_color]
# pos = nx.spring_layout(G, k=1000/np.sqrt(G.order()), weight="weight", iterations=1000)
# nx.draw_networkx(G, pos=pos, edge_color="orange", edge_cmap=edg_color, width=[i/60 for i in edg_color], font_color="white", font_size=8, node_size=200)
# plt.show()

In [11]:
print("Running on whole time and merging to big df...")
metrics_full = {}
df_pop, eu_iso = g.load_population_df()
big_df_col = ["year","month","prod","country"] +  ['edges','nodes','size','degree',
    'degree_perc',
    'out_degree',
    'out_degree_perc',
    'in_degree',
    'in_degree_perc',
    'density',
    'density_adj',
    'clustering',
    'page_rank',
    'vulnerability',
    'in_weight_degree',
    'out_weight_degree',
    'degree_gamma',
    'degree_xmin',
    'in_degree_gamma',
    'in_degree_xmin',
    'out_degree_gamma',
    'out_degree_xmin',
    'in_weight_degree_gamma',
    'in_weight_degree_xmin',
    'out_weight_degree_gamma',
    'out_weight_degree_xmin',
    'out_weight_abs',
    'in_weight_abs']

big_df = pd.DataFrame(columns=big_df_col)
issues = []
for y in tqdm([y for y in range(2001,2021)]):
    # print(y,m)
    df = g.load_filtered_data("complete", save=True, force_reload=True, verbose=False, sorted=False, columns=g.full_columns, types=g.full_types, group_by_prod_code=True, n_digits=2,
                                years=[y], months=None, product="all", flow="all", declarant_iso="all", partner_iso="all", trade_type="all")
    n4g = g.get_world_countries(df_pop, eu_iso, year=str(y))
    for prod in tqdm(df.PRODUCT_CPA2_1.unique(),leave=False):
        df4g = df[df.PRODUCT_CPA2_1 == prod]
        t4g = g.extract_table_for_graph(df4g, y=str(y), flow="all", scale_by='population', pop_df=df_pop)
        if t4g.shape[0] < 10:
            issues.append((y,prod,t4g.shape[0]))
            # print(y,m,prod,t4g.shape[0])
        coord, metrics, G = g.makeGraph(t4g, tab_nodes=None, weight_flag=True, criterio="VALUE_IN_EUROS")
        nx.write_gexf(G,f"data-samples/graphs/complete/complete_y{y}_p{prod}.gexf")
        x = metrics.reset_index().to_dict()
        x["year"] = str(y)
        x["month"] = "-"
        x["prod"] = prod
        big_df = pd.concat([big_df,pd.DataFrame(x,columns=big_df_col)],axis=0)
        metrics_full[f"{y}_cpa_{prod}"] = metrics #.sort_values("degree",ascending=False)
        if prod > "10":
            break
    break
print(issues)
# with open("./data-samples/metrics/issues_complete.txt","w") as f:
#     f.write(str(issues))
# pickle.dump(metrics_full,open("./data-samples/manual/metrics/metrics_complete_V_2.pickle","wb"))
# big_df.to_parquet("./data-samples/manual/metrics/metrics_complete_V_2.parquet")


Running on whole time and merging to big df...



[]


In [12]:
# df_com, df_wto, df_y = load_filtered_data("complete", save=True, force_reload=False, verbose=False, sorted=False, columns=g.full_columns, types=g.full_types, group_by_prod_code=True, n_digits=2,
#                                 years=[2007], months=None, product="all", flow="all", declarant_iso="all", partner_iso="all", trade_type="all")

In [13]:
# df_y = g.combine_sources(df_com,df_wto)

In [14]:
# df_y[(df_y == "").any(axis=1)]

In [15]:
# df_full = load_filtered_data("full", save=True, force_reload=False, verbose=False, sorted=False, columns=g.full_columns, types=g.full_types, group_by_prod_code=True, n_digits=2,
#                                  years=[2007], months=None, product="all", flow="all", declarant_iso="all", partner_iso="all", trade_type="all")

In [16]:
# df_full = pd.read_parquet("data-samples/full/Years/full200752.parquet")[g.full_columns]

In [17]:
# df_try = pd.concat([df_full[(df_full == "").any(axis=1)] for i in range(10)])

In [18]:
# df_try = df_full.loc[6675030:6675040]

In [19]:
# df_try.loc[(df_try == "").any(axis=1),"QUANTITY_IN_KG"] = 0

In [20]:
# df_try.loc[df_try[(df_try == "").any(axis=1)].index,"QUANTITY_IN_KG"] = 0

In [21]:
# for y in trange(2007,2021):
#     df = pd.read_parquet(f"data-samples/full/Years/full{y}52.parquet")
#     l = df[df.QUANTITY_IN_KG == ""].index.to_list()
#     df.loc[l,"QUANTITY_IN_KG"] = 0
#     # df.loc[l,"QUANTITY_IN_KG"]
#     df.astype(g.full_types).to_parquet(f"data-samples/full/Years/full{y}52.parquet")

In [22]:
# df[df.QUANTITY_IN_KG == ""]